<a href="https://colab.research.google.com/github/GuidoGiacomoMussini/Born_rule_Image_Classification/blob/main/ResNet_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Model
from keras.datasets import cifar10

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Concatenate training and test data
X = np.concatenate((x_train, x_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0).flatten()

# Initialize the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Define intermediate models to extract features from different blocks
layers_of_interest = ['conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']
models = [Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output) for layer in layers_of_interest]

# Preprocess the images for ResNet50
X_preprocessed = preprocess_input(X)

# Check if GPU is available
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# Extract and save features for each layer
feature_dfs = {}
for layer_name, model in zip(layers_of_interest, models):
    features = model.predict(X_preprocessed, batch_size=64)
    features_list = [features[i] for i in range(features.shape[0])]
    df = pd.DataFrame({
        'image': list(X.squeeze()),  # Assuming images are in grayscale; modify if RGB
        'descriptors': features_list,
        'label': y
    })
    feature_dfs[layer_name] = df

    df.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/Universita/Tesi/Script_Definitivi/cifar10_ResNet_features_{layer_name}.pkl')
    print(f"layer {layer_name} saved")

GPU is available
938/938 [==============================] - 6s 4ms/step
layer conv2_block3_out saved
938/938 [==============================] - 6s 6ms/step
layer conv3_block4_out saved
938/938 [==============================] - 15s 15ms/step
layer conv4_block6_out saved
938/938 [==============================] - 11s 10ms/step
layer conv5_block3_out saved


In [ ]:
df.to_pickle('/content/drive/MyDrive/Colab Notebooks/Universita/Tesi/Script_Definitivi/cifar10_ResNet_Features.pkl')